# mapping-challenge-mask_rcnn-training
![CrowdAI-Logo](https://github.com/crowdAI/crowdai/raw/master/app/assets/images/misc/crowdai-logo-smile.svg?sanitize=true)

This notebook contains the baseline code for the training a vanilla [Mask RCNN](https://arxiv.org/abs/1703.06870) model for the [crowdAI Mapping Challenge](https://www.crowdai.org/challenges/mapping-challenge).

This code is adapted from the [Mask RCNN]() tensorflow implementation available here : [https://github.com/matterport/Mask_RCNN](https://github.com/matterport/Mask_RCNN).

First we begin by importing all the necessary dependencies : 

In [1]:
import os
import sys
import time
import numpy as np

# Download and install the Python COCO tools from https://github.com/waleedka/coco
# That's a fork from the original https://github.com/pdollar/coco with a bug
# fix for Python 3.
# I submitted a pull request https://github.com/cocodataset/cocoapi/pull/50
# If the PR is merged then use the original repo.
# Note: Edit PythonAPI/Makefile and replace "python" with "python3".
#  
# A quick one liner to install the library 
# !pip install git+https://github.com/waleedka/coco.git#subdirectory=PythonAPI

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

from mrcnn.evaluate import build_coco_results, evaluate_coco
from mrcnn.dataset import MappingChallengeDataset

import zipfile
import urllib.request
import shutil


## Dataset location 
Now we have to download all the files in the datasets section and untar them to have the following structure :
```
├── data
|   ├── pretrained_weights.h5 (already included in this repository)
│   ├── test
│   │   └── images/
│   │   └── annotation.json
│   ├── train
│   │   └── images/
│   │   └── annotation.json
│   └── val
│       └── images/
│       └── annotation.json
```
Note that the `pretrained_weights.h5` (available at [https://www.crowdai.org/challenges/mapping-challenge/dataset_files](https://www.crowdai.org/challenges/mapping-challenge/dataset_files)) are the weights used for the baseline submission, and are obtained by running the learning schedule mentioned later in the experiment. In the said experiment, the initial weights used can be found [here](https://github.com/matterport/Mask_RCNN/releases/download/v2.1/mask_rcnn_balloon.h5). 

In [2]:
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils


PRETRAINED_MODEL_PATH = os.path.join(ROOT_DIR,"data/" "pretrained_weights.h5")
LOGS_DIRECTORY = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


## Experiment Configuration

In [3]:
class MappingChallengeConfig(Config):
    """Configuration for training on data in MS COCO format.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = "crowdai-mapping-challenge"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 5

    # Uncomment to train on 8 GPUs (default is 1)
    GPU_COUNT = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # 1 Backgroun + 1 Building

#     STEPS_PER_EPOCH=90
#     VALIDATION_STEPS=20

    IMAGE_MAX_DIM=320
    IMAGE_MIN_DIM=320
    
    MEAN_PIXEL = [81.16231469, 86.53528546, 64.72005973]

config = MappingChallengeConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     40
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      8
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 5
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  320
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  320
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [320 320   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_mask_loss': 1.0, 'rpn_class_loss': 1.0, 'mrcnn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_bbox_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE            

## Instantiate Model

In [4]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=LOGS_DIRECTORY)
# Load pretrained weights
model_path = PRETRAINED_MODEL_PATH
model.load_weights(model_path, by_name=True)

Instructions for updating:
Colocations handled automatically by placer.


## Load Training and Validation Dataset

In [5]:
# Load training dataset
dataset_train = MappingChallengeDataset()
dataset_train.load_dataset(dataset_dir=os.path.join("data", "train"))
dataset_train.prepare()

# Load validation dataset
dataset_val = MappingChallengeDataset()
val_coco = dataset_val.load_dataset(dataset_dir=os.path.join("data", "val"))
dataset_val.prepare()

Annotation Path  data/train/annotation.json
Image Dir  data/train/images
loading annotations into memory...
Done (t=3.05s)
creating index...
index created!
Annotation Path  data/val/annotation.json
Image Dir  data/val/images
loading annotations into memory...
Done (t=0.51s)
creating index...
index created!


## Train

In [6]:
# *** This training schedule is an example. Update to your needs ***

# Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=4,
            layers='heads')

# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=12,
            layers='4+')

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=16,
            layers='all')

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/yoninachmany/crowdai-mapping-challenge-mask-rcnn/logs/crowdai-mapping-challenge20190428T1951/mask_rcnn_crowdai-mapping-challenge_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed

/home/yoninachmany/.local/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/yoninachmany/.local/lib/python3.5/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/4
 856/1000 [========================>.....] - ETA: 3:23 - loss: 2.4008 - rpn_class_loss: 0.2467 - rpn_bbox_loss: 1.0289 - mrcnn_class_loss: 0.2917 - mrcnn_bbox_loss: 0.3836 - mrcnn_mask_loss: 0.4499

 860/1000 [========================>.....] - ETA: 3:17 - loss: 2.3994 - rpn_class_loss: 0.2462 - rpn_bbox_loss: 1.0283 - mrcnn_class_loss: 0.2915 - mrcnn_bbox_loss: 0.3835 - mrcnn_mask_loss: 0.4498

 862/1000 [========================>.....] - ETA: 3:14 - loss: 2.3986 - rpn_class_loss: 0.2460 - rpn_bbox_loss: 1.0280 - mrcnn_class_loss: 0.2914 - mrcnn_bbox_loss: 0.3835 - mrcnn_mask_loss: 0.4497

...

 927/1000 [==========================>...] - ETA: 2:00 - loss: 1.6775 - rpn_class_loss: 0.1326 - rpn_bbox_loss: 0.6578 - mrcnn_class_loss: 0.2410 - mrcnn_bbox_loss: 0.2612 - mrcnn_mask_loss: 0.3849

 930/1000 [==========================>...] - ETA: 1:55 - loss: 1.6772 - rpn_class_loss: 0.1325 - rpn_bbox_loss: 0.6577 - mrcnn_class_loss: 0.2410 - mrcnn_bbox_loss: 0.2611 - mrcnn_mask_loss: 0.3849

 931/1000 [==========================>...] - ETA: 1:53 - loss: 1.6771 - rpn_class_loss: 0.1325 - rpn_bbox_loss: 0.6576 - mrcnn_class_loss: 0.2410 - mrcnn_bbox_loss: 0.2611 - mrcnn_mask_loss: 0.3849

 971/1000 [============================>.] - ETA: 47s - loss: 1.6783 - rpn_class_loss: 0.1323 - rpn_bbox_loss: 0.6581 - mrcnn_class_loss: 0.2415 - mrcnn_bbox_loss: 0.2612 - mrcnn_mask_loss: 0.3852

1000/1000 [==============================] - 1695s 2s/step - loss: 1.6829 - rpn_class_loss: 0.1325 - rpn_bbox_loss: 0.6599 - mrcnn_class_loss: 0.2425 - mrcnn_bbox_loss: 0.2621 - mrcnn_mask_loss: 0.3859 - val_loss: 2.6937 - val_rpn_class_loss: 0.3374 - val_rpn_bbox_loss: 1.0235 - val_mrcnn_class_loss: 0.3387 - val_mrcnn_bbox_loss: 0.4883 - val_mrcnn_mask_loss: 0.5057
Epoch 14/16
1000/1000 [==============================] - 1317s 1s/step - loss: 1.6476 - rpn_class_loss: 0.1237 - rpn_bbox_loss: 0.6438 - mrcnn_class_loss: 0.2386 - mrcnn_bbox_loss: 0.2568 - mrcnn_mask_loss: 0.3846 - val_loss: 2.7588 - val_rpn_class_loss: 0.3386 - val_rpn_bbox_loss: 1.1612 - val_mrcnn_class_loss: 0.2805 - val_mrcnn_bbox_loss: 0.4789 - val_mrcnn_mask_loss: 0.4995
Epoch 15/16
1000/1000 [==============================] - 1323s 1s/step - loss: 1.6178 - rpn_class_loss: 0.1201 - rpn_bbox_loss: 0.6251 - mrcnn_class_loss: 0.2392 - mrcnn_bbox_loss: 0.2527 - mrcnn_mask_loss: 0.3807 - val_loss: 2.9880 - val_rpn_class_l

Now you can monitor the training by running : 
```
tensorboard --logdir=logs/[path-to-your-experiment-logdir]
```
and if everything works great, you should see something like : 
![loss-plot](images/loss-plot.png)

# Author
Sharada Mohanty [sharada.mohanty@epfl.ch](sharada.mohanty@epfl.ch)